In [1]:
import os
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from ignite.metrics import Rouge
from pandas.core.common import random_state
from PIL import Image , ImageFilter
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer, VisionEncoderDecoderModel, ViTImageProcessor , BertTokenizer
from typing import Any, Dict, List

c:\Users\amtom\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 48763

np.random.seed(seed=seed, )

random_state(state=seed, )

random.seed(a=seed, )

torch.manual_seed(seed=seed, )
torch.cuda.manual_seed(seed=seed, )
torch.cuda.manual_seed_all(seed=seed, )
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia", "bert-base-cased").to(device)
######################################################## bert based model
tokenizer = BertTokenizer.from_pretrained(############## bert based model
    "bert-base-cased" , cache_dir= '/cache'
)

print(tokenizer.all_special_tokens)

tokenizer.add_special_tokens({
    "bos_token": "[beginoftext]",
    "pad_token": "[PAD]",
    "eos_token": "[EOS]",
})

tokenizer.bos_token_id = 400404

model.config.decoder_start_token_id = tokenizer.bos_token_id
model.generation_config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.decoder.resize_token_embeddings(len(tokenizer))
processor = ViTImageProcessor.from_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia" , cache_dir='/cache')

c:\Users\amtom\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of ViTModel were not initialized from the model checkpoint at nickmuchi/vit-finetuned-chest-xray-pneumonia and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [4]:
class CXIRGDataset(Dataset):
    def __init__(self, data: List[Dict[str, Any]]) -> None:
        super(CXIRGDataset, self).__init__()
        self.data = data

    def __getitem__(self, index: int) -> Dict[str, Any]:
        return self.data[index]

    def __len__(self) -> int:
        return len(self.data)

In [5]:
max_length = 256

def train_dl_collate_fn(one_batch_data: List[Dict[str, Any]]):
    names = [one_data["name"] for one_data in one_batch_data]

    pixel_values = processor(
        images=[one_data["image"] for one_data in one_batch_data], 
        return_tensors="pt"
    ).pixel_values

    label_ids = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=[
            (tokenizer.bos_token + one_data["text"] + tokenizer.eos_token) for one_data in one_batch_data
        ],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).input_ids

    return names, pixel_values.to(device), label_ids.to(device)

def valid_dl_collate_fn(one_batch_data: List[Dict[str, Any]]):
    names = [one_data["name"] for one_data in one_batch_data]

    pixel_values = processor(
        images=[one_data["image"] for one_data in one_batch_data], 
        return_tensors="pt"
    ).pixel_values

    label_ids = tokenizer.batch_encode_plus(
        batch_text_or_text_pairs=[
            (tokenizer.bos_token + one_data["text"] + tokenizer.eos_token) for one_data in one_batch_data
        ],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    ).input_ids

    return names, pixel_values.to(device), label_ids.to(device)

In [6]:
train_data = []

report_path = "data/train_data/reports.xlsx"
report_df = pd.read_excel(report_path)

image_dir_path = "data/train_data/images"
for image_name in os.listdir(image_dir_path):
    image = Image.open(os.path.join(image_dir_path, image_name))
    image= image.filter(ImageFilter.DETAIL)############################################################ additional operation: image filter
    image_array = np.array(image).astype(np.float32)################################################### additional operation: normalize
    normalized_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))
    image = Image.fromarray((normalized_array * 255).astype(np.uint8))
    if image.mode != "RGB":
        image = image.convert("RGB")

    text = report_df[report_df["name"] == image_name[:13]]["text"].values[0].replace("_x000D_", "\r")

    train_data.append({
        "name": image_name[:13],
        "image": image,
        "text": text
    })

train_dataset = CXIRGDataset(train_data)

In [7]:
valid_data = []

report_path = "data/valid_data/reports.xlsx"
report_df = pd.read_excel(report_path)

image_dir_path = "data/valid_data/images"
for image_name in os.listdir(image_dir_path):
    image = Image.open(os.path.join(image_dir_path, image_name))
    image= image.filter(ImageFilter.DETAIL)############################################################ additional operation: image filter
    image_array = np.array(image).astype(np.float32)################################################### additional operation: normalize
    normalized_array = (image_array - np.min(image_array)) / (np.max(image_array) - np.min(image_array))
    image = Image.fromarray((normalized_array * 255).astype(np.uint8))
    if image.mode != "RGB":
        image = image.convert("RGB")

    text = report_df[report_df["name"] == image_name[:13]]["text"].values[0].replace("_x000D_", "\r")

    valid_data.append({
        "name": image_name[:13],
        "image": image,
        "text": text
    })

valid_dataset = CXIRGDataset(valid_data)

In [8]:
lr = 1e-5
epochs = 30
optimizer = AdamW(params=model.parameters(), lr=lr)

train_batch_size = 1
valid_batch_size = 1
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=train_dl_collate_fn
)
valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=valid_batch_size,
    shuffle=False,
    collate_fn=valid_dl_collate_fn
)

rouge = Rouge(variants=["L", 2], multiref="best")

In [9]:
def evaluate(model: VisionEncoderDecoderModel, epoch: int) -> Dict[str, float]:
    model.eval()

    pbar = tqdm(valid_dataloader)
    pbar.set_description(f"Evaluting Epoch: {epoch + 1}")

    with torch.no_grad():
        for names, pixel_values, label_ids in pbar:
            predictions = model.generate(pixel_values)

            _predictions = tokenizer.batch_decode(
                predictions,
                skip_special_tokens=True
            )

            _labels = tokenizer.batch_decode(
                label_ids,
                skip_special_tokens=True
            )

            print(f"Names      : {names}")
            print(f"Predictions: {_predictions}")
            print(f"Labels     : {_labels}")
            print()

            for _prediction, _label in zip(_predictions, _labels):
                split_prediction = _prediction.split()
                split_label = _label.split()

                rouge.update(([split_prediction], [[split_label]]))

    return rouge.compute()

In [10]:
def generate_result(model: VisionEncoderDecoderModel, epoch: int) -> Dict[str, float]:
    results = []
    model.eval()

    pbar = tqdm(valid_dataloader)
    pbar.set_description(f"Evaluting Epoch: {epoch + 1}")

    with torch.no_grad():
        for names, pixel_values, label_ids in pbar:
            predictions = model.generate(pixel_values)

            _predictions = tokenizer.batch_decode(
                predictions,
                skip_special_tokens=True
            )

            _labels = tokenizer.batch_decode(
                label_ids,
                skip_special_tokens=True
            )

            print(f"Names      : {names}")
            print(f"Predictions: {_predictions}")
            print(f"Labels     : {_labels}")
            results.append(_predictions)
    return _predictions

In [11]:
min_loss = float("inf")
rouge_result = []
for epoch in range(epochs):
    model.train()

    pbar = tqdm(train_dataloader)
    pbar.set_description(f"Training Epoch [{epoch + 1} / {epochs}]")

    lost_list = []
    average_loss = 0

    for _, pixel_values, label_ids in pbar:
        optimizer.zero_grad()

        loss = model(
            pixel_values=pixel_values,
            labels=label_ids
        ).loss

        lost_list.append(loss.item())
        average_loss = np.mean(np.array(lost_list))
        loss.backward()
        optimizer.step()

        pbar.set_postfix(loss=average_loss)

    if average_loss < min_loss:
        min_loss = average_loss
        torch.save(model.state_dict(), f"outputs/best_checkpoint.pth")
    result = evaluate(model=model, epoch=epoch)
    rouge_result.append(result)
    print(f"Rouge-2 score on epoch {epoch}:", result)

Evaluting Epoch: 1:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\amtom\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Evaluting Epoch: 1:  10%|█         | 1/10 [00:00<00:04,  1.89it/s]

Names      : ['NLP_CHEST_002']
Predictions: ['']
Labels     : ['Chest PA view shows : Partial atelectasis at left lower lung field. Left pleural effusion. Borderline heart size. Atherosclerotic change of aortic knob. S / P port - A implantation via left subclavian vein. Intraperitoneal port catheter. Surgical clips at RUQ of abdomen.']



Evaluting Epoch: 1:  20%|██        | 2/10 [00:01<00:03,  2.02it/s]

Names      : ['NLP_CHEST_004']
Predictions: ['']
Labels     : ['Chest PA view show : Impression : - S / P RLL wedge resection. - Bilateral lungs metastasis. - Left lower lung subsegmental atelectasis. - Increased infiltrations in both lungs. - Blunting right CP angle. - Tortuous atherosclerotic aorta. - Scoliosis, DJD and osteoporosis of spine. - Compression fracture of L1.']



Evaluting Epoch: 1:  30%|███       | 3/10 [00:01<00:03,  2.06it/s]

...

Names      : ['NLP_CHEST_059']
Predictions: ['Chest AP view showed : 1. Enlarged heart size with tortuous']
Labels     : ['Supine chest AP view shows : Pigtail catheter at right lower chest. Decreased amount of right pleural effusion. Air in the right lower pleural space. Poor lung expansion. Mild subcutaneous emphysema at right chest wall. Right pleural thickening. Ill - defined masses at right perihilar region. Normal heart size.']



Evaluting Epoch: 30:  90%|█████████ | 9/10 [00:04<00:00,  2.03it/s]

Names      : ['NLP_CHEST_071']
Predictions: ['Chest plain film shows : Impression : - Suspicious pulmonary edema']
Labels     : ['Chest film shows : Impression : - Bilateral lung infiltrations. - Suspect right lower lung patch. Blunting right CP angle. - Tortuous atherosclerotic aorta. - Scoliosis, DJD and osteoporosis of spine. Compression fracture of T12. Old fracture of left ribs. - S / P fixation in L - spine. - S / P tracheostomy and NG tube.']



Evaluting Epoch: 30: 100%|██████████| 10/10 [00:04<00:00,  2.05it/s]

Names      : ['NLP_CHEST_085']
Predictions: ['Chest plain film view showed : Impression : - Suspicious pulmonary ed']
Labels     : ['Chest X ray : - Right pneumothorax with pleural effusion, status post drainge tube placement. - Extensive subcutaneous emphysema from neck to right side thoracic cage. - Increased right hilar opacity. - Atherosclerosis and tortuous aorta. - Obscured bilateral costophrenic angles. - Spondylosis and mild scoliosis of thoracolumbar spine. - OA of right glenohumeral joint. - Generalized osteopenia.']

Rouge-2 score on epoch 29: {'Rouge-L-P': 0.4750545565545566, 'Rouge-L-R': 0.11573167150500115, 'Rouge-L-F': 0.11573167150500115, 'Rouge-2-P': 0.3107457047309993, 'Rouge-2-R': 0.06608814849376543, 'Rouge-2-F': 0.06608814849376543}


In [21]:
import pandas as pd
results = pd.DataFrame(rouge_result)
results

,Rouge-L-P,Rouge-L-R,Rouge-L-F,Rouge-2-P,Rouge-2-R,Rouge-2-F
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.027778,0.010676,0.010676,0.002941,0.000735,0.000735
2,0.146022,0.041770,0.041770,0.055788,0.012775,0.012775
3,0.232774,0.057831,0.057831,0.121260,0.023490,0.023490
4,0.279716,0.070337,0.070337,0.147523,0.030518,0.030518
5,0.311127,0.077427,0.077427,0.159299,0.032998,0.032998
6,0.337742,0.084072,0.084072,0.180374,0.037441,0.037441
7,0.351566,0.086682,0.086682,0.186943,0.038436,0.038436
8,0.368109,0.088393,0.088393,0.203983,0.040665,0.040665
9,0.378132,0.090362,0.090362,0.211262,0.042102,0.042102


In [12]:
#model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("nickmuchi/vit-finetuned-chest-xray-pneumonia", "bert-base-cased").to(device)
#model.load_state_dict(torch.load('outputs/best_checkpoint.pth'))

In [13]:
results = []
model.eval()

pbar = tqdm(valid_dataloader)
pbar.set_description(f"Evaluting Epoch: {epoch + 1}")

with torch.no_grad():
    for names, pixel_values, label_ids in pbar:
        predictions = model.generate(pixel_values)

        _predictions = tokenizer.batch_decode(
                predictions,
                skip_special_tokens=True
        )
        results.append(_predictions)

Evaluting Epoch: 30: 100%|██████████| 10/10 [00:04<00:00,  2.11it/s]


In [14]:
results

[['Chest PA view shows : Impression : - Increased both lung markings.'],
 ['Chest plain film shows : Impression : - Suspicious pulmonary edema'],
 ['Chest AP view showed : 1. Enlarged heart size with tortuous'],
 ['Chest PA view shows : Large mass occupying right upper chest. Normal heart size.'],
 ['Chest plain film view showed : Impression : - Suspicious pulmonary ed'],
 ['Chest PA view : Impression : - Increased both lung markings. -'],
 ['Chest AP view showed : 1. Normal heart size with tortuous aort'],
 ['Chest AP view showed : 1. Enlarged heart size with tortuous'],
 ['Chest plain film shows : Impression : - Suspicious pulmonary edema'],
 ['Chest plain film view showed : Impression : - Suspicious pulmonary ed']]

In [15]:
import csv

# List of strings

# Open or create a CSV file in write mode
with open('submission.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write all strings in a single row
    writer.writerow(results)

print("Data has been written to output.csv")

Data has been written to output.csv
